## Society of Minds Assignment 2
Here's how the following code works:-
* User submits the claim: It goes to inner verification team for verification.
* Fact_Checker_Agent confirms the claim against any reliable news
* Cross-Referencer agent confirms the work done by the fact checker
* Human-in-the-loop makes the final decision on wheteher the fact is correct or not
* If it is incorrect the flow stops
* If it is correct the flow goes to summarizer agent to prepare a summary of the claim and verification
* Again the human agent is prompted to either approve the draft or suggest edits or reject it
* Finally, if it is approved by the human agent it goes to the publisher agent to publish the report


In [1]:
from autogen_agentchat.agents import AssistantAgent
import asyncio
import os
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.models import ModelInfo
from dotenv import load_dotenv

In [2]:
model_info = ModelInfo(
    model="gemini-2.5-pro",
    is_chat_model=True,
    is_function_calling_model=True ,
    vision=False,
    function_calling=True,
    json_output=True,
    family='unknown',
    multiple_system_messages=True
      # Set to False if the model doesn't support function calling
)

In [3]:
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
model_client = OpenAIChatCompletionClient(model='gemini-2.5-pro',api_key=api_key, model_info=model_info)
my_assistant = AssistantAgent(name='Assistant',model_client=model_client)

c:\Users\mukun\anaconda3\envs\autogen\lib\site-packages\autogen_ext\models\openai\_openai_client.py:439: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [4]:

fact_checker = AssistantAgent(
    name="fact_checker",
    model_client=model_client,
    system_message=(
        "You are a fact checker. Given a news claim, check its internal consistency and identify "
        "the main factual claims that must be verified. Output 'VERIFIED' or 'REJECTED' at the end "
        "if you can conclusively verify or must reject; otherwise list the missing / questionable facts."
    ),
)

cross_ref = AssistantAgent(
    name="cross_ref",
    model_client=model_client,
    system_message=(
        "You are a cross-reference agent. For the listed claims, attempt to cross-check against "
        "trusted sources and say 'VERIFIED' if evidence supports the claims or 'REJECTED' if widely contradicted. "
        "If uncertain, list the checks that failed or need human review."
    ),
)
termination1 = TextMentionTermination("VERIFIED")
termination2 = TextMentionTermination("REJCTED")
verification_termination = termination1 | termination2

verification_inner_team = RoundRobinGroupChat(
    [fact_checker, cross_ref],
    termination_condition=verification_termination,
    max_turns=6,
)


In [5]:
verification_society_agent = SocietyOfMindAgent(
    name="verification_society_agent",
    model_client=model_client,
    team=verification_inner_team,
    response_prompt=(
        "From the verification discussion above, output a concise result. "
        "Start with either 'VERIFIED' or 'REJECTED', followed by a short list of key verified facts "
        "or the reasons for rejection. Output only that."
    ),
)

In [6]:
summarizer_agent = AssistantAgent(
    name="summarizer_agent",
    model_client=model_client,
    system_message=(
        "You are a news summarizer. Given verified facts, write a short, "
        "publish-ready news paragraph with a clear headline line first. Keep it to 30 words."
        "Do not publish yet."
    ),
)

In [7]:
human_editor_agent = UserProxyAgent(
    name="human_editor_agent",
    description=(
        """
        You are the human editor. Review the proposed article and either:\n
        - Type APPROVE to accept and publish as-is
        - Type REJECT to reject
        - Or provide edit instructions (e.g. 'Refine language', 'Add source citation', or paste an edited draft')"""
    ),
)

In [8]:
publisher_agent = AssistantAgent(
    name="publisher_agent",
    model_client=model_client,
    system_message=(
        "You are the publisher assistant. Given the final approved article text, produce a single-line "
        "publish message (headline + 1 sentence) suitable for the website. Keep it under 30 words."
    ),
)

In [9]:
async def collect_final_stream_message(async_gen):

    last_msg = None
    async for msg in async_gen:
        try:
            print(f"[{getattr(msg, 'source', 'agent')}] {getattr(msg, 'content', getattr(msg, 'chat_message', msg))}")
        except Exception:
            print(msg)
        last_msg = msg
    return last_msg

In [10]:
def get_message_text(msg):
    
    if msg is None:
        return ""
 
    content = None
    if hasattr(msg, "content"):
        content = msg.content
    elif hasattr(msg, "chat_message") and hasattr(msg.chat_message, "content"):
        content = msg.chat_message.content
    else:
        content = str(msg)
    
    if isinstance(content, list):
        content = " ".join(str(x) for x in content)
    return str(content)

In [ ]:
async def main(task_text):

    # Stage 1: Verification inner team (wrapped by verification_society_agent)

    ver_stream = verification_society_agent.run_stream(task=task_text)
    final_ver_msg = await collect_final_stream_message(ver_stream)
    final_ver_text = get_message_text(final_ver_msg).strip()
    print("\nVERIFICATION RESULT RAW:\n", final_ver_text, "\n")

    if final_ver_text.upper().startswith("REJECTED"):
        print("\nVerification failed. Stopping pipeline. Human editor and publisher will not run.")
        return

    if not final_ver_text.upper().startswith("VERIFIED"):
        print("\nVerification inconclusive. Asking human editor for decision.")
        # If verification is inconclusive, we can choose to send entire verification output to human for override
        # we ask human agent to decide
        human_input = f"VERIFICATION INCONCLUSIVE:\n{final_ver_text}"
     
        human_msg = await human_editor_agent.run(task=human_input)
        human_text = get_message_text(human_msg).strip()
        if "APPROVE" not in human_text.upper():
            print("\nHuman did not approve. Pipeline stops.")
            return
        else:

            approved_facts = human_text
    else:

        parts = final_ver_text.split(None, 1)
        approved_facts = final_ver_text if len(parts) == 1 else parts[1].strip()

    #Stage 2: Summarization & Human edit
    print("\nSTAGE 2: Summarization")
    
    summ_stream = summarizer_agent.run_stream(task=approved_facts)
    final_summ_msg = await collect_final_stream_message(summ_stream)
    final_summ_text = get_message_text(final_summ_msg).strip()
    print("\nSUMMARIZER DRAFT:\n", final_summ_text, "\n")


    print("\nStage 2b: Human Editor Review")
   
    human_stream = human_editor_agent.run_stream(task=final_summ_text)
    final_human_msg = await collect_final_stream_message(human_stream)
    final_human_text = get_message_text(final_human_msg).strip()
    print("\nHUMAN AGENT RESPONSE:\n", final_human_text, "\n")

    if "APPROVE" not in final_human_text.upper():
        print("\nHuman rejected or requested edits.")
        return

    # If human approved, pass the final article to publisher
    print("\nStage 3: Publishing")
    # If human provided an edited draft, prefer that; else use summarizer draft
    final_article_text = final_human_text if final_human_text.strip() and final_human_text.upper() != "APPROVE" else final_summ_text

    pub_stream = publisher_agent.run_stream(task=final_article_text)
    final_pub_msg = await collect_final_stream_message(pub_stream)
    final_pub_text = get_message_text(final_pub_msg).strip()
    print("\nPUBLISH MESSAGE:\n", final_pub_text, "\n")
    print("Workflow complete")


In [ ]:
task = """ 
        News: Virat Kohli to not play in the 2027 ODI world cup.
        Claim:Virat Kohli will soon announce his retirement from ODI cricket.
        Source: social media post from a sports journalist. Verify and prepare a short publishable paragraph if verified.
        """

In [16]:
await main(task)

[user]  
        News: Virat Kohli to n ot play n the 2027 ODI world cup.
        Claim:Virat Kohli will sson announce his retirement from ODI cricket.
        Source: social media post from a sports journalist. Verify and prepare a short publishable paragraph if verified.
        
[fact_checker] Based on the information provided, this claim cannot be verified and appears to be speculative.

### Fact-Checking Analysis:

*   **Internal Consistency:** The news ("will not play in the 2027 World Cup") and the claim ("will soon announce retirement from ODIs") are internally consistent. One would be the direct result of the other. However, consistency does not equal factual accuracy.
*   **Source Credibility:** The source is cited as "a social media post from a sports journalist." This is a weak source without further details. The credibility of the journalist and their track record for breaking accurate news is paramount and is currently unknown. High-profile player retirements are almost a

In [12]:
task = """ 
        On July 1, 2024, the Government of India officially rolled out the UPI Lite X feature for offline payments,
         enabling transactions without an internet connection, as announced by the Reserve Bank of India.
        """

In [13]:
await main(task)

[user]  
        On July 1, 2024, the Government of India officially rolled out the UPI Lite X feature for offline payments,
         enabling transactions without an internet connection, as announced by the Reserve Bank of India.
        
[fact_checker] This claim contains a mix of correct and incorrect information.

### Internal Consistency Check:

The claim is internally consistent. It is plausible that a government would roll out a financial feature that was previously announced by its central bank. The components of the claim do not contradict each other.

### Main Factual Claims to be Verified:

1.  A feature named "UPI Lite X" for offline payments exists.
2.  It was officially rolled out by the Government of India.
3.  The rollout date was July 1, 2024.
4.  The feature was announced by the Reserve Bank of India (RBI).

### Fact-Checking Analysis:

1.  **Claim 1 (Existence of UPI Lite X):** **CORRECT.** UPI Lite X is a real feature developed by the National Payments Corporation o